In [1]:
import pandas as pd
import glob
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import os
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

In [27]:
# coding: utf-8

# In[1]:


# rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys, RDKFingerprint, Descriptors, rdMolDescriptors, Lipinski
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import rdFingerprintGenerator

# sklearn
from sklearn import svm
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, \
    recall_score, cohen_kappa_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

# typing
from typing import Union

# connecting to chembl through web client
from chembl_webresource_client.new_client import new_client

from datetime import datetime
import shutil

from datetime import datetime
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import time

from joblib import dump
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split

# In[2]:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier



In [2]:
csv_list = glob.glob('*_sub_location.csv')
csv_list

['drug_central_sub_location.csv',
 'chembl_sub_location.csv',
 'pharos_sub_location.csv']

In [3]:
df = pd.concat([pd.read_csv(csv_file) for csv_file in csv_list])
df.dropna(subset=['location'],inplace=True)
df

,uni_prot_id,location
0,O43172,"Nucleus, Nucleus speckle"
1,O95342,Apical cell membrane Multi-pass membrane prote...
2,P68363,Cytoplasm Cytoskeleton
4,Q401N2,Cell membrane Multi-pass membrane protein
5,Q53GL7,"Nucleus Nucleolus, Cytoplasm"
...,...,...
952,Q9Y5Y4,Cell membrane Multi-pass membrane protein
953,P26439,Endoplasmic reticulum membrane Single-pass mem...
954,P36888,"Membrane Single-pass type I membrane protein, ..."
955,Q5JUK3,Cell membrane Multi-pass membrane protein


In [4]:
df[df['location'].str.contains('synapse')].loc[101]['location']

'Cell membrane Peripheral membrane protein, Membrane Clathrin-coated pit, Presynapse'

In [5]:
df[~df['location'].str.contains('synapse')]

,uni_prot_id,location
0,O43172,"Nucleus, Nucleus speckle"
1,O95342,Apical cell membrane Multi-pass membrane prote...
2,P68363,Cytoplasm Cytoskeleton
4,Q401N2,Cell membrane Multi-pass membrane protein
5,Q53GL7,"Nucleus Nucleolus, Cytoplasm"
...,...,...
952,Q9Y5Y4,Cell membrane Multi-pass membrane protein
953,P26439,Endoplasmic reticulum membrane Single-pass mem...
954,P36888,"Membrane Single-pass type I membrane protein, ..."
955,Q5JUK3,Cell membrane Multi-pass membrane protein


In [6]:
df_synapse = df[df['location'].str.contains('synapse')]

In [7]:
def all_synapse(s):
    words = s.split(',')
    return all('synapse' in word.lower() for word in words)

df_synapse.loc[df_synapse['location'].apply(all_synapse), 'location'] = np.nan
df_synapse

/tmp/ipykernel_266459/2594861034.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_synapse.loc[df_synapse['location'].apply(all_synapse), 'location'] = np.nan


,uni_prot_id,location
101,Q2M2I8,"Cell membrane Peripheral membrane protein, Mem..."
94,O15056,"Cytoplasm, Cell membrane, Membrane raft, Presy..."
471,Q09470,"Cell membrane Multi-pass membrane protein, Mem..."
496,P42261,"Cell membrane Multi-pass membrane protein, End..."
1353,P35626,NaN
1380,Q9C0H9,"Cytoplasm, Cytoplasm Cytoskeleton, Cell projec..."
1392,Q2M2I8,"Cell membrane Peripheral membrane protein, Mem..."
1694,P28223,"Cell membrane Multi-pass membrane protein, Cel..."
1958,P78352,"Cell membrane Lipid-anchor Cytoplasmic side, P..."
2556,P21554,"Cell membrane Multi-pass membrane protein, Mem..."


In [8]:
df_pharos = pd.read_csv('pharos.csv')
df_pharos = df_pharos[['smiles','uniprot']]
df_pharos

,smiles,uniprot
0,CCCCC1(CCCC)CN(C2=CC=CC=C2)C2=C(C=C(OCC(=O)N[C...,Q12908
1,CC(C)(CO)C1=CC2=C(C=C(F)C(NC(=O)C3(CC3)C3=CC=C...,P13569
2,NaN,P13569
3,CC(C)(C)C1=CC(=C(NC(=O)C2=CNC3=CC=CC=C3C2=O)C=...,P13569
4,CC1=CC=C(NC(=O)C2(CC2)C2=CC=C3OC(F)(F)OC3=C2)N...,P13569
...,...,...
3862,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,Q16513
3863,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,Q9UHD2
3864,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,O00506
3865,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9P289


In [9]:
df_chembl = pd.read_csv('chembl.csv')
df_chembl = df_chembl[['SMILES','UniProt_ID']]
df_chembl = df_chembl.rename(columns={'SMILES': 'smiles', 'UniProt_ID': 'uniprot'})
df_chembl

,smiles,uniprot
0,CC(=N)NCCSCC[C@H](N)C(=O)O,P35228
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P20309
2,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P08172
3,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,Q92731
4,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,P03372
...,...,...
394540,CCn1c(C(=O)N(C2CC2)C2CC2)cc2c3c(ncn3C)c(Nc3cc(...,Q05397
394541,CCn1c(C(=O)N(C2CC2)C2CC2)cc2c3c(ncn3C)c(Nc3cc(...,P07949
394542,CCn1c(C(=O)N(C2CC2)C2CC2)cc2c3c(ncn3C)c(Nc3cc(...,P12931
394543,CCn1c(C(=O)N(C2CC2)C2CC2)cc2c3c(ncn3C)c(Nc3cc(...,Q9Y6E0


In [10]:
df_drug_central = pd.read_csv('drug_central/tchem_drugs_05122020.tsv',sep='\t')
df_drug_central = df_drug_central[['smiles','uniprot']]
df_drug_central

,smiles,uniprot
0,COC1=C(OCCCN2CCOCC2)C=CC2=C1N=C(NC(=O)C1=CN=C(...,P42338
1,O=C1NC2=C(C=C1)C=CC(OCCCCN1CCN(CC1)C1=C3C=CSC3...,P21917
2,O=C1NC2=C(C=C1)C=CC(OCCCCN1CCN(CC1)C1=C3C=CSC3...,P50406
3,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](COP(O)(=O)OP(O...,P51575
4,NC1=NC=NC2=C1N=CN2[C@@H]1O[C@H](COP(O)(=O)OP(O...,Q99571
...,...,...
638,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9Y6E0
639,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q8IVH8
640,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,P0DMS8
641,CCN(C1CCOCC1)C1=CC(=CC(C(=O)NCC2=C(C)C=C(C)NC2...,Q92800


In [11]:
df_all = pd.concat([df_chembl, df_pharos, df_drug_central])
df_all = (
    df_all.dropna(subset=['smiles'])
    .drop_duplicates()
    .merge(df, left_on='uniprot', right_on='uni_prot_id', how='left')
    .drop_duplicates()
)
mask = (df_all['uni_prot_id'] != 'P35626') & (df_all['location'].notna()) & (~df_all['location'].str.contains('synapse', case=False, na=False))
df_all = df_all[mask]
df_all['location'] = df_all['location'].str.replace(', ', ',').str.lower().str.split(',')

In [12]:
df_all = pd.concat([df_chembl,df_pharos,df_drug_central])
df_all.dropna(subset=['smiles'],inplace=True)
df_all.drop_duplicates(inplace=True)
df_all = df_all.merge(df, left_on='uniprot', right_on='uni_prot_id', how='left')
df_all.drop_duplicates(inplace = True)
df_all = df_all[df_all['uni_prot_id']!='P35626']
df_all.dropna(subset=['location'],inplace=True)
df_all = df_all[df_all['location']!= np.nan]
df_all = df_all[~df_all['location'].str.contains('synapse', case=False)]
df_all['location'] = df_all['location'].str.replace(', ', ',').str.lower()
df_all['location'] = df_all['location'].str.split(',')
df_all

,smiles,uniprot,uni_prot_id,location
0,CC(=N)NCCSCC[C@H](N)C(=O)O,P35228,P35228,[cytoplasm cytosol]
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P20309,P20309,"[cell membrane multi-pass membrane protein, po..."
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P08172,P08172,"[cell membrane multi-pass membrane protein, po..."
5,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,Q92731,Q92731,[nucleus]
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,P03372,P03372,"[nucleus, cytoplasm, cell membrane peripheral ..."
...,...,...,...,...
675170,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9P289,Q9P289,"[cytoplasm, golgi apparatus]"
675173,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9Y6E0,Q9Y6E0,"[cytoplasm, nucleus, membrane]"
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,P0DMS8,P0DMS8,[cell membrane multi-pass membrane protein]
675179,CCN(C1CCOCC1)C1=CC(=CC(C(=O)NCC2=C(C)C=C(C)NC2...,Q92800,Q92800,[nucleus]


In [13]:
df_smiles = pd.DataFrame(df_all['smiles'])

# If you want 'smiles' as the column name in df_smiles
df_smiles.columns = ['smiles']
df_smiles.drop_duplicates(inplace=True)
df_smiles.to_csv('dataset1.csv', index=False)

In [14]:
old_label=False
file_name = 'new-label-all-final-LV.csv'
if old_label:
    df_label = pd.read_csv(file_name,header=None)
else:
    df_label = pd.read_csv(file_name,header=None)
    df_label = df_label.T.reset_index(drop=True)

df_label = df_label.applymap(lambda s: s.lower() if type(s) == str else s)

label_dict = {}
for label_index, row in enumerate(df_label.itertuples(index=False)):
    for location in row:
        if not isinstance(location, float):
            label_dict[location] = label_index

label_dict = {key.strip(): value for key, value in label_dict.items()}

In [15]:
df_label.to_csv('label-all-new.csv',index=False, header=False)

In [16]:
def replace_from_dict(lst, replace_dict):
    return [replace_dict.get(item, item) for item in lst]

In [17]:
df_all['location'] = df_all['location'].apply(replace_from_dict, args=(label_dict,))
df_all

,smiles,uniprot,uni_prot_id,location
0,CC(=N)NCCSCC[C@H](N)C(=O)O,P35228,P35228,[5]
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P20309,P20309,"[12, 12, 12, 8]"
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P08172,P08172,"[12, 12]"
5,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,Q92731,Q92731,[nucleus]
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,P03372,P03372,"[nucleus, 5, 12, nucleus, 5, 12, 12, nucleus, ..."
...,...,...,...,...
675170,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9P289,Q9P289,"[5, golgi apparatus]"
675173,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9Y6E0,Q9Y6E0,"[5, nucleus, 23]"
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,P0DMS8,P0DMS8,[12]
675179,CCN(C1CCOCC1)C1=CC(=CC(C(=O)NCC2=C(C)C=C(C)NC2...,Q92800,Q92800,[nucleus]


In [18]:
df_all['location'] = df_all['location'].apply(lambda lst: [x for x in lst if not isinstance(x, str)])
df_all

,smiles,uniprot,uni_prot_id,location
0,CC(=N)NCCSCC[C@H](N)C(=O)O,P35228,P35228,[5]
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P20309,P20309,"[12, 12, 12, 8]"
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P08172,P08172,"[12, 12]"
5,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,Q92731,Q92731,[]
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,P03372,P03372,"[5, 12, 5, 12, 12, 23]"
...,...,...,...,...
675170,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9P289,Q9P289,[5]
675173,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9Y6E0,Q9Y6E0,"[5, 23]"
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,P0DMS8,P0DMS8,[12]
675179,CCN(C1CCOCC1)C1=CC(=CC(C(=O)NCC2=C(C)C=C(C)NC2...,Q92800,Q92800,[]


In [19]:
df_all = df_all[df_all['location'].apply(lambda x: len(x) > 0)]
df_all

,smiles,uniprot,uni_prot_id,location
0,CC(=N)NCCSCC[C@H](N)C(=O)O,P35228,P35228,[5]
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P20309,P20309,"[12, 12, 12, 8]"
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,P08172,P08172,"[12, 12]"
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,P03372,P03372,"[5, 12, 5, 12, 12, 23]"
9,CN(C)C[C@@H]1CCn2cc(c3ccccc32)C2=C(C(=O)NC2=O)...,P24723,P24723,[5]
...,...,...,...,...
675167,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,O00506,O00506,[5]
675170,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9P289,Q9P289,[5]
675173,CCOC1=CC2=NC=C(C#N)C(NC3=CC=C(OCC4=CC=CC=N4)C(...,Q9Y6E0,Q9Y6E0,"[5, 23]"
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,P0DMS8,P0DMS8,[12]


In [20]:
df_all.drop(['uniprot','uni_prot_id'],axis=1,inplace=True)
df_all['location'] = df_all['location'].apply(tuple)
df_all.drop_duplicates(inplace=True)
df_all

/tmp/ipykernel_266459/3082369734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.drop(['uniprot','uni_prot_id'],axis=1,inplace=True)
/tmp/ipykernel_266459/3082369734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['location'] = df_all['location'].apply(tuple)
/tmp/ipykernel_266459/3082369734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.drop_dup

,smiles,location
0,CC(=N)NCCSCC[C@H](N)C(=O)O,"(5,)"
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"(12, 12, 12, 8)"
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"(12, 12)"
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,"(5, 12, 5, 12, 12, 23)"
9,CN(C)C[C@@H]1CCn2cc(c3ccccc32)C2=C(C(=O)NC2=O)...,"(5,)"
...,...,...
675156,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(2,)"
675159,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(5, 12, 13)"
675162,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(5, 23, 4, 13, 1)"
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,"(12,)"


In [21]:
df_final = df_all[['smiles','location']]
df_final.to_csv('final_with_smiles_location_LV.csv',index=False)
df_final

,smiles,location
0,CC(=N)NCCSCC[C@H](N)C(=O)O,"(5,)"
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"(12, 12, 12, 8)"
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"(12, 12)"
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,"(5, 12, 5, 12, 12, 23)"
9,CN(C)C[C@@H]1CCn2cc(c3ccccc32)C2=C(C(=O)NC2=O)...,"(5,)"
...,...,...
675156,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(2,)"
675159,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(5, 12, 13)"
675162,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(5, 23, 4, 13, 1)"
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,"(12,)"


In [22]:
max_values = df_final['location'].apply(lambda x: max(x) if isinstance(x, tuple) else None)
max_value = max_values.max()
max_value

26

In [33]:
vector_size = max_value
def one_hot_encode_location(loc_tuple):
    vector = [0] * vector_size

    for num in loc_tuple:
        if 0 < num <= vector_size:
            vector[num - 1] = 1  
    return np.array(vector)

df_final['location_encoded'] = df_final['location'].apply(one_hot_encode_location)
    
df_final

,smiles,location,location_encoded,Molecule,RDKFingerprint,np_RDKFingerprint
0,CC(=N)NCCSCC[C@H](N)C(=O)O,"(5,)","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f371b7f5dd0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"(12, 12, 12, 8)","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f371b7f5cf0>,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"(12, 12)","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f371b7f5b30>,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,"(5, 12, 5, 12, 12, 23)","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f371b7f5ba0>,"[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, ...","[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, ..."
9,CN(C)C[C@@H]1CCn2cc(c3ccccc32)C2=C(C(=O)NC2=O)...,"(5,)","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f371b7f5c10>,"[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, ..."
...,...,...,...,...,...,...
675156,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(2,)","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f3686289200>,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
675159,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(5, 12, 13)","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f3686289270>,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
675162,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"(5, 23, 4, 13, 1)","[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f36862892e0>,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,"(12,)","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",<rdkit.Chem.rdchem.Mol object at 0x7f3686289350>,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, ...","[1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, ..."


In [37]:
np.vstack(df_final['location_encoded'].values).shape

(150966, 26)

In [25]:
df_final[['smiles','location_encoded']]

,smiles,location_encoded
0,CC(=N)NCCSCC[C@H](N)C(=O)O,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ..."
3,C[C@@H]1O[C@H](C[N+](C)(C)C)C[C@H]1O,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
7,O=C1c2ccccc2C(=O)c2c1ccc(O)c2O,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
9,CN(C)C[C@@H]1CCn2cc(c3ccccc32)C2=C(C(=O)NC2=O)...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
675156,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
675159,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ..."
675162,[H][C@]12C[C@]([H])([C@@H](OC)[C@](C)(O1)N1C3=...,"[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
675176,COC1=CC(NC2=NC(NC3=CC=C4OC(C)(C)C(=O)N(COP(O)(...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [28]:
def apply_descriptors_and_fp(df: pd.DataFrame,
                             label_column: str,
                             smiles_column: str,
                             fp_method: str,
                             use_fingerprint: bool = True,
                             use_descriptors: bool = True) -> Union[np.ndarray, np.ndarray]:
    """Applies descriptor and fingerprint methods to a DataFrame and returns scaled descriptors and labels as numpy arrays.

    Args:
        df (pd.DataFrame): DataFrame containing the SMILES strings and labels.
        label_column (str): Name of the label column.
        smiles_column (str): Name of the SMILES column.
        fp_method (str): Name of the fingerprint method to apply.
        use_descriptors (bool): Whether or not to include descriptors in the output.
        use_fingerprint (bool): Whether or not to include fingerprint in the output.

    Returns:
        X (np.ndarray): Descriptor and/or fingerprint values depending on use_descriptors and use_fingerprint flags.
        Y (np.ndarray): Labels.
    """
    descriptor_methods_dict = {
        'MolWt': Descriptors.MolWt,
        'HeavyAtomCount': Descriptors.HeavyAtomCount,
        'NumHDonors': Descriptors.NumHDonors,
        'NumHAcceptors': Descriptors.NumHAcceptors,
        'MolLogP': Descriptors.MolLogP,
        'TPSA': rdMolDescriptors.CalcTPSA,
        'NumRotatableBonds': Lipinski.NumRotatableBonds,
        'NumAromaticRings': Descriptors.NumAromaticRings,
        'NumAliphaticRings': Descriptors.NumAliphaticRings,
    }
    fp_methods_dict = {
        'RDKFingerprint': RDKFingerprint,
        'MorganFingerprint': lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2),
        'MACCSkeys': MACCSkeys.GenMACCSKeys,
        'AvalonFP': pyAvalonTools.GetAvalonFP,
        'Gobbi-Paalman': rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect,
    }

    # Validate the fingerprint method
    if fp_method not in fp_methods_dict:
        raise ValueError(f"Invalid fingerprint method. Choices are: {list(fp_methods_dict.keys())}")

    # Convert SMILES to RDKit molecules
    df['Molecule'] = df[smiles_column].apply(Chem.MolFromSmiles)

    # Apply the descriptor methods and scale them if use_descriptors is True
    if use_descriptors:
        for descriptor, method in descriptor_methods_dict.items():
            df[descriptor] = df['Molecule'].apply(method)
        scaler = MinMaxScaler()
        df[list(descriptor_methods_dict.keys())] = scaler.fit_transform(df[list(descriptor_methods_dict.keys())])

    # Apply the fingerprint method if use_fingerprint is True
    if use_fingerprint:
        df[fp_method] = df['Molecule'].apply(fp_methods_dict[fp_method])
        df[f'np_{fp_method}'] = df[fp_method].apply(
            lambda fingerprints: np.array(list(map(int, fingerprints.ToBitString()))))
        X_fp = np.stack(df[f'np_{fp_method}'].values)

    # Create numpy array for descriptor data if use_descriptors is True
    if use_descriptors and use_fingerprint:
        X_descr = df[list(descriptor_methods_dict.keys())].values
        # Concatenate along columns axis (assuming fingerprint is 1D)
        X = np.hstack((X_descr, X_fp))
    elif use_descriptors:
        X = df[list(descriptor_methods_dict.keys())].values
    elif use_fingerprint:
        X = X_fp
    else:
        raise ValueError("At least one of use_descriptors or use_fingerprint must be True")

    # Create numpy array for label data
    if label_column:
        Y = df[label_column].values
    else:
        Y = None

    return X, Y, df

In [30]:
X, Y, df = apply_descriptors_and_fp(
                                    df_final,
                                    'location_encoded',
                                    'smiles',
                                    'RDKFingerprint',
                                    True,
                                    False)

In [32]:
numpy_array = np.array(Y)
numpy_array

array([list([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       ...,
       list([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
      dtype=object)